In [12]:
import pandas as pd
import numpy as np
import glob
import os
from tqdm import tqdm
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from sklearn.model_selection import train_test_split

In [15]:
def create_features(seg_id, seg, X):
    xc = pd.Series(seg['acoustic_data'].values)
    zc = np.fft.fft(xc)
    
    X.loc[seg_id, 'mean'] = xc.mean()
    X.loc[seg_id, 'std'] = xc.std()
    X.loc[seg_id, 'max'] = xc.max()
    X.loc[seg_id, 'min'] = xc.min()

In [16]:
submission = pd.read_csv('LANL-Earthquake-Prediction/sample_submission.csv')
submission.shape

X_test = pd.DataFrame(index=range(submission.shape[0]), dtype=np.float32)

# Load files located in the test directory
test_files = glob.glob(os.path.join('LANL-Earthquake-Prediction/test', '*_*.*'))

# Feature engineering for the test set
# Iterate over all files in the test directory and
# create the test dataframe with new features
all_test_files = len(test_files)
#all_test_files =600
for seg_id in tqdm(range(all_test_files)):
    seg = pd.read_csv(test_files[seg_id])
    create_features(seg_id, seg, X_test)

100%|██████████| 2624/2624 [01:16<00:00, 34.17it/s]


In [24]:
model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)
train_labels = list(range(len(X_test)))
# Fit model
model.fit(X_test, train_labels)
# Get predictions
preds = model.predict(X_test)

0:	learn: 756.6505426	total: 636us	remaining: 636us
1:	learn: 756.1128236	total: 1.23ms	remaining: 0us


In [25]:
preds

array([1328.60085383, 1328.60085383, 1328.60085383, ..., 1252.04388484,
       1328.60085383, 1328.60085383])